In [15]:
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import scipy.stats as stats

In [16]:
#Portafolio
Tickers=['CEMEXCPO.MX','CUERVO.MX','LACOMERUBC.MX','GRUMAB.MX','VOLARA.MX']
Titulos=[800,200,250,100,700]
if len(Tickers) != len(Titulos):
    raise ValueError("¡Error! Las listas Tickers y Titulos deben tener la misma longitud.")
Nro_de_obs='504d'
intervalo_tiempo='1d'
if len(Tickers)!=len(Titulos):
    print('El nro. de títulos no coincide con el nro. de activos')

In [17]:
#Extracción de precios de Yahoo Finance
def get_data(symbol:str):
    data=yf.download(tickers=symbol,period=Nro_de_obs,interval=intervalo_tiempo)
    data.reset_index(inplace=True)
    return data
dataframes=[] #lista que contendrá dataframes como elementos
df_final=pd.DataFrame()

for Ticker in Tickers:
    data=get_data(Ticker)
    #Renombrar las columnas por cada ticker
    data=data.set_axis(['Date',f'Close_{Ticker}','High','Low','Open','Volume'],axis=1)

    if df_final.empty:
        df_final=data[['Date',f'Close_{Ticker}']]
    else:
        df_final=pd.merge(df_final,data[['Date',f'Close_{Ticker}']],on='Date',how='outer')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [18]:
#Calcular los rendimientos de cada activo, con los precios de cierre del dataframe
for Ticker in Tickers:
    df_final[f'Rend_{Ticker}']=df_final[f'Close_{Ticker}']/df_final[f'Close_{Ticker}'].shift(1)-1 #Shift toma el dato anterior

In [19]:
#Código para el Backtesting
df_final["VaR histórico 99%"]=np.nan
df_final["(%) VaR histórico 99%"]=np.nan
df_final["Valuacion"]=np.nan
#definimos el tamaño de la ventana sobre la que se calcula el VaR
#(Nro. de ob. totales) - (Nro. de obs. sobre las que se hará el backtesting) - (Primera fila (no contiene info de los rendimientos))
ventana=len(df_final)-251-1
for k in range (1,252+1): #Nro. de calculos de VaR histórico que se harán, es decir 252
    vector=[]
    for j in range(1,ventana+1):
        portafolio=0
        i=0
        for Ticker in Tickers:
            PrecioUlt=df_final[f"Close_{Ticker}"].iloc[-253+k]
            PrecioSim=PrecioUlt*(1+df_final[f"Rend_{Ticker}"].iloc[-252-j+k])
            Ganancia_Perdida=(PrecioSim-PrecioUlt)*Titulos[i]
            portafolio+=Ganancia_Perdida
            i+=1
        vector.append(portafolio)
        
    df_final.at[df_final.index[-253 + k], "VaR histórico 99%"]=pd.Series(vector).quantile(0.01)
    
    valuacion=0
    i=0
    for Ticker in Tickers:
            PrecioUlt=df_final[f"Close_{Ticker}"].iloc[-253+k]
            Ganancia_Perdida=PrecioUlt*Titulos[i]
            valuacion+=Ganancia_Perdida
            i+=1
    df_final.at[df_final.index[-253 + k], "(%) VaR histórico 99%"]=df_final["VaR histórico 99%"].iloc[-253+k]/valuacion
    df_final.at[df_final.index[-253 + k], "Valuacion"]=valuacion

df_final["VaR histórico 99%"]=df_final["VaR histórico 99%"].shift(1)
df_final["(%) VaR histórico 99%"]=df_final["(%) VaR histórico 99%"].shift(1)
df_final
            
        
        
    
    
    
    

,Date,Close_CEMEXCPO.MX,Close_CUERVO.MX,Close_LACOMERUBC.MX,Close_GRUMAB.MX,Close_VOLARA.MX,Rend_CEMEXCPO.MX,Rend_CUERVO.MX,Rend_LACOMERUBC.MX,Rend_GRUMAB.MX,Rend_VOLARA.MX,VaR histórico 99%,(%) VaR histórico 99%,Valuacion
0,2023-04-12,10.178427,44.396103,40.137894,259.299988,20.510000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-04-13,10.716229,44.435123,40.384624,255.800110,22.709999,0.052838,0.000879,0.006147,-0.013497,0.107265,NaN,NaN,NaN
2,2023-04-14,10.766026,43.742500,39.476665,249.438477,21.740000,0.004647,-0.015587,-0.022483,-0.024870,-0.042712,NaN,NaN,NaN
3,2023-04-17,10.815823,43.410824,39.753002,247.746536,21.889999,0.004625,-0.007582,0.007000,-0.006783,0.006900,NaN,NaN,NaN
4,2023-04-18,10.636555,42.396278,39.141113,251.062698,21.420000,-0.016575,-0.023371,-0.015392,0.013385,-0.021471,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2025-04-07,10.640000,19.510000,35.799999,359.000000,10.200000,0.016237,-0.024012,0.007032,-0.007382,0.024096,-2429.039542,-0.037718,64403.999996
500,2025-04-08,10.610000,19.850000,36.490002,366.989990,9.840000,-0.002820,0.017427,0.019274,0.022256,-0.035294,-2430.225518,-0.037734,65167.499352
501,2025-04-09,10.880000,19.950001,36.959999,365.230011,10.370000,0.025448,0.005038,0.012880,-0.004796,0.053862,-2452.841051,-0.037639,65716.001034
502,2025-04-10,10.250000,20.139999,36.119999,368.000000,9.700000,-0.057904,0.009524,-0.022727,0.007584,-0.064609,-2475.926573,-0.037676,64847.999477


In [20]:
df_final["Perd/Gan"]=np.nan
df_final["(%) Perd/Gan"]=np.nan

for k in range(1,252):
    df_final.at[df_final.index[-252+k],"Perd/Gan"]=df_final["Valuacion"].iloc[-252+k]-df_final["Valuacion"].iloc[-253+k]
    df_final.at[df_final.index[-252+k],"(%) Perd/Gan"]=df_final["Valuacion"].iloc[-252+k]/df_final["Valuacion"].iloc[-253+k]-1

df_final["Excepciones VaR histórico 99%"]=(df_final["(%) VaR histórico 99%"]>df_final["(%) Perd/Gan"]).astype(int)

In [24]:
Eficiencia_VaR_historico=df_final["Excepciones VaR histórico 99%"].sum()/252

In [25]:
print(Eficiencia_VaR_historico)

0.9760956175298805


In [ ]:
def ordenamiento(df, columna):
    col = columna
    cols = [c for c in df.columns if c != col] + [col]
    return df[cols]

for columna in ["Valuacion", "Perd/Gan", "(%) Perd/Gan", "VaR histórico 99%", "(%) VaR histórico 99%", "Excepciones VaR histórico 99%"]:
    df_final = ordenamiento(df_final, columna)

In [23]:
ruta_excel = r"C:\Users\denil\Downloads\df_final_final2.xlsx"

# Exportar el DataFrame a Excel
df_final.to_excel(ruta_excel, index=False)

In [9]:
dfpl=df_final[:]
#Calcular los parámetros de la dist Normal Teórica
media=dfpl['Portafolio'].mean()
volatilidad=dfpl['Portafolio'].std()

#Crear el prefijo para mi gráfico
fig=go.Figure()

#Gráfico
fig.add_trace(go.Histogram(x=dfpl['Portafolio'],
                           nbinsx=50, #nbins es el nro. de barritas
                           marker_color='skyred',
                           opacity=0.75,name='Datos Reales'))

#Generar la dist normal teórica mediante generar un rango en el eje X
#Tal rango va de la pérdida máxima a la menor y lo particiona en 100
#Después grafica esos valores al evaluarlos en una N(media,volatilidad)
x=np.linspace(dfpl['Portafolio'].min(),dfpl['Portafolio'].max(),100)
y=stats.norm.pdf(x,media,volatilidad)

#Escalamiento para la función de distribución teórica 
bin_pdf=(dfpl['Portafolio'].max()-dfpl['Portafolio'].min())/50
y_scaled=y*bin_pdf*len(dfpl)
#Añadirla al gráfico anterior de barras
fig.add_trace(go.Scatter(x=x,
                         y=y_scaled,
                         mode='lines',
                         name='Distribución Normal',
                         line=dict(color='green',width=2,dash='dash')))

#Añadir una línea verticar que señale el VaR
fig.add_shape(go.layout.Shape(type='line',
                              x0=VaR_99,
                              y0=0,
                              x1=VaR_99,
                              y1=max(y_scaled),
                              line=dict(color='red',width=3,dash='dash')))

#Mostrar Gráfico
fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'color' property of histogram.marker
        Received value: 'skyred'

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deeps